In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## Loading the data

데이터에 존재하는 태그 파악하여 딕셔너리 생성

In [2]:
tags = {'O': 0, 'B-PS': 1, 'I-PS': 2, 'B-LC': 3, 'I-LC': 4,
        'B-OG': 5, 'I-OG': 6, 'B-QT': 7, 'I-QT': 8, 'B-DT': 9,
        'I-DT': 10, 'B-TI': 11, 'I-TI': 12, 'B-LT': 13, 'I-LT': 14,
        'B-PO': 15, 'I-PO': 16
        }
tag_size = len(tags)
tag_size

17

In [3]:
print(tags) # <- tag_to_index

{'O': 0, 'B-PS': 1, 'I-PS': 2, 'B-LC': 3, 'I-LC': 4, 'B-OG': 5, 'I-OG': 6, 'B-QT': 7, 'I-QT': 8, 'B-DT': 9, 'I-DT': 10, 'B-TI': 11, 'I-TI': 12, 'B-LT': 13, 'I-LT': 14, 'B-PO': 15, 'I-PO': 16}


In [4]:
index_to_tag = {j : i for i,j in tags.items()}
print(index_to_tag)

{0: 'O', 1: 'B-PS', 2: 'I-PS', 3: 'B-LC', 4: 'I-LC', 5: 'B-OG', 6: 'I-OG', 7: 'B-QT', 8: 'I-QT', 9: 'B-DT', 10: 'I-DT', 11: 'B-TI', 12: 'I-TI', 13: 'B-LT', 14: 'I-LT', 15: 'B-PO', 16: 'I-PO'}


load_data 함수를 정의 : 데이터를 불러와 문장, 태그(labels) 구분하여 저장

In [5]:
def load_data(file_path):
  texts, labels = [], []

  with open(file_path, 'r', encoding = 'utf-8') as file:
    text, label = [], []
    for line in file:
      if line.startswith('##') or not line.strip():
        if text:
          texts.append(" ".join(text))
          labels.append([tags[i] for i in label])
          text, label = [], []
        continue
      parts = line.strip().split('\t')
      if len(parts) != 2:
        continue
      char, tag = parts
      text.append(char)
      label.append(tag)

    return texts, labels

train, dev data에서 각각 text, label을 구분하여 저장

In [6]:
train_texts, train_labels = load_data('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/NLP/Group Project/Phase 3/Data/klue_train.tsv')
dev_texts, dev_labels = load_data('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/NLP/Group Project/Phase 3/Data/klue_dev.tsv')

In [7]:
train_texts[1:3]

['한 군 데 서 필 름 을 너 무 낭 비 한 작 품 입 니 다 .', '하 지 만 이 영 화 에 는 감 히 별 5 개 를 주 고 싶 다']

In [8]:
train_labels[1:3]

[[7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0]]

In [9]:
print(f"train data는 {len(train_texts)}개 있고, dev data는 {len(dev_texts)}개 있답니다!")

train data는 21008개 있고, dev data는 5000개 있답니다!


## Tokenize, Padding

In [10]:
import torch
from transformers import BertTokenizer, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader

In [11]:
class PIIDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]


        encoding = self.tokenizer.encode_plus(text,
                                              add_special_tokens = True,
                                              max_length = self.max_length,
                                              return_token_type_ids = False,
                                              padding = 'max_length',
                                              truncation = True,
                                              return_attention_mask = True,
                                              return_tensors = 'pt'
                                              )

        tokens = self.tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

        temp_labels = [0] * len(tokens)

        j = 0
        for i, token in enumerate(tokens):
          if token.startswith("##"):
            continue
          if token in ['[CLS]','[SEP]','[PAD]']:
            temp_labels[i] = 0
          elif j < len(label):
            temp_labels[i] = label[j]
            j += 1

        return {
            'input_ids' : encoding['input_ids'].flatten(),
            'attention_mask' : encoding['attention_mask'].flatten(),
            'labels' : torch.tensor(temp_labels, dtype = torch.long)
        }

## Dataset, DataLoader

In [12]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
max_length = 128

# Dataset
train_data = PIIDataset(train_texts, train_labels, tokenizer, max_length)
dev_data = PIIDataset(dev_texts, dev_labels, tokenizer, max_length)

# DataLoader
train_dataloader = DataLoader(train_data, batch_size = 32, shuffle = True)
dev_dataloader = DataLoader(dev_data, batch_size = 32, shuffle = False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [13]:
model = BertForTokenClassification.from_pretrained('klue/bert-base', num_labels = tag_size)
model.to(device)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

## Model Fitting

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir = './results',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 1e-5, # 기존 2e-5에서 수정
    per_device_train_batch_size = 16, # 기존 32에서 수정
    per_device_eval_batch_size = 16, # 기존 32에서 수정
    num_train_epochs = 7, # 기존 5에서 수정
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 10,
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss',
    greater_is_better = False
)

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience = 3,
    early_stopping_threshold = 0.0
)

trainer = Trainer(
    model = model,
    args = training_args,
    callbacks = [early_stopping_callback],
    train_dataset = train_data,
    eval_dataset = dev_data
)

trainer.train()
trainer.evaluate()

model.save_pretrained('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/NLP/Group Project/Phase 4/')
tokenizer.save_pretrained('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/NLP/Group Project/Phase 4/')

print(" 모델 저장 완료! :D ")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.023900,0.060161
2,0.019000,0.060861
3,0.010000,0.061998
4,0.012200,0.063986


('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/NLP/Group Project/Phase 3/Trials/tokenizer_config.json',
 '/content/drive/MyDrive/2. KOREA UNIV./KUBIG/NLP/Group Project/Phase 3/Trials/special_tokens_map.json',
 '/content/drive/MyDrive/2. KOREA UNIV./KUBIG/NLP/Group Project/Phase 3/Trials/vocab.txt',
 '/content/drive/MyDrive/2. KOREA UNIV./KUBIG/NLP/Group Project/Phase 3/Trials/added_tokens.json')

## Loading the Model

In [14]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/NLP/Group Project/Phase 4/')
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

## Functions for new Sentences

In [15]:
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random

### ner_pii_masking

----------------
< ner_pii_masking >
- 가장 기본이 되는 함수
- 모든 label 마스킹, [MASK] 로 출력
----------------

In [28]:
def ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device):

  # 숫자 정보 마스킹 후 리스트 num에 저장 (순서대로) ---------------------------
  numbers = {
      r'\d{6}-\d{7}' : '[RRN]',

      r'\d{3}-\d{2}-\d{6,7}' : '[ACC]',
      r'\d{3}-\d{3}-(\d{6}|\d{8})' : '[ACC]',
      r'\d{3}-(\d{5}|\d{8})-\d{3}' : '[ACC]',
      r'\d{3}-\d{6}-\d{5}' : '[ACC]',
      r'\d{3}-\d{4}-\d{4}-\d{2}' : '[ACC]',
      r'\d{3}-\d{5}-\d{3}-\d{2}' : '[ACC]',
      r'\d{3}-\d{6}-\d{2}-\d{3}' : '[ACC]',
      r'\d{4}-\d{2,3}-\d{6,7}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}-\d{1}' : '[ACC]',
      r'\d{1}-\d{6}-\d{2,3}' : '[ACC]',
      r'(\d{2}|\d{6})-\d{2}-\d{6}' : '[ACC]',

      r'\d{3}-\d{4}-\d{4}' : '[TEL]',
      r'\d{2}-\d{3,4}-\d{4}' : '[TEL]'
  }

  sorted_patterns = sorted(numbers.items(), key=lambda x: len(x[0]), reverse=True)

  num = []
  matches = []
  already_matched = set()

  for pattern, token in sorted_patterns:
    for match in re.finditer(pattern, sentence):
      start = match.start()
      end = match.end()
      if all(pos not in already_matched for pos in range(start, end)):
        matches.append((match.start(), token))
        already_matched.update(range(start, end))


  matches.sort(key = lambda x : x[0])

  num = [token for _, token in matches]

  # 숫자 정보를 제외한 문자 정보만으로 input 만든 뒤 모델에 넣어 ---------------

  pattern = '|'.join(f'({key})' for key in numbers.keys())

  sentence_no_num = re.sub(pattern, '[PAD]', sentence) # 숫자 정보는 [PAD] 으로 처리

  tokenized_sentence = tokenizer.tokenize(sentence_no_num)

  sent_in_num = tokenizer.convert_tokens_to_ids(tokenized_sentence)

  input_sentence = pad_sequences([sent_in_num], maxlen = max_length, dtype = 'int',
                                    value = 0, truncating = 'post', padding = 'post')

  input_mask = [[1 if tok > 0 else 0 for tok in sent] for sent in input_sentence]

  input_sentence = torch.tensor(input_sentence).to(device)
  input_mask = torch.tensor(input_mask).to(device)

  with torch.no_grad():
    output = model(input_sentence, attention_mask = input_mask)
    logits = output.logits

  pred = np.argmax(logits.cpu().numpy(), axis = 2)

  tokens = tokenizer.convert_ids_to_tokens(input_sentence[0])

  # 다시 문장의 형태로 결합 ----------------------------------------------------

  out_sentence = []
  current_token = ""
  prev_tag = None
  tag_name = None

  puncs = [".",","]

  for token, tag_index in zip(tokens, pred[0]):
      if (token == '[PAD]'):
        if current_token:
          out_sentence.append(current_token)
          current_token = ""
        out_sentence.append('[PAD]')

      elif (tag_index == 0):
        if token in puncs:
          current_token += token

        elif token.startswith("##"):
          current_token += (token[2:])

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (token)

      elif (tag_index > 0):
        tag_name = index_to_tag[tag_index][2:]

        if token.startswith("##"):
          if tag_name == prev_tag:
            pass
          else:
            current_token += (f"[MASK-{tag_name}]")

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (f"[MASK-{tag_name}]")

      prev_tag = tag_name

  out_sentence.append(current_token)

  out_sentence = ' '.join(out_sentence)

  # [PAD] -> 마스킹 토큰으로 ---------------------------------------------------

  out_sentence_split = out_sentence.split()

  new_sentence = []

  pad_count = 0

  for token in out_sentence_split:
    if token == '[PAD]':
      if pad_count < len(num):
        new_sentence.append(num[pad_count])
        pad_count += 1
    else:
      new_sentence.append(token)

  out_sentence = ' '.join(new_sentence)

  return(out_sentence)

In [ ]:
sentence = "이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

이비[MASK-PS]는 우리 집 막내이며, [MASK-LC] [MASK-LC]에 살고 있다. 전화번호는 [TEL] 이고, 주민등록번호는 [RRN] 이다. 또한, 우리 [ACC] 은행을 쓴다.


In [17]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

고양이 [MASK-PS]는 우리 집 막내이며, [MASK-LC] [MASK-LC]에 살고 있다. 전화번호는 [TEL] 이고, 주민등록번호는 [RRN] 이다. 또한, 우리 [ACC] 은행을 쓴다.
이 문장에는 ['[TEL]', '[RRN]', '[ACC]']의 숫자 데이터가 있습니다.
문장을 다시 결합하기 전에는 ['고양이', '이비', '##비', '##는', '우리', '집', '막내', '##이', '##며', ',', '서울', '강남구', '##에', '살', '##고', '있', '##다', '.', '전화', '##번', '##호', '##는', '[PAD]', '이', '##고', ',', '주민', '##등', '##록', '##번', '##호', '##는', '[PAD]', '이다', '.', '또한', ',', '우리', '[PAD]', '은행', '##을', '쓴다', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

### ner_pii_masking_loose

----------------
< ner_pii_masking_loose >
- 루즈한 마스킹 모델
- 이름, 계좌번호, 주민등록번호, 전화번호 마스킹
----------------

In [27]:
def ner_pii_masking_loose(sentence, model, tokenizer, index_to_tag, max_length, device):

  # 숫자 정보 마스킹 후 리스트 num에 저장 (순서대로) ---------------------------
  numbers = {
      r'\d{6}-\d{7}' : '[RRN]',

      r'\d{3}-\d{2}-\d{6,7}' : '[ACC]',
      r'\d{3}-\d{3}-(\d{6}|\d{8})' : '[ACC]',
      r'\d{3}-(\d{5}|\d{8})-\d{3}' : '[ACC]',
      r'\d{3}-\d{6}-\d{5}' : '[ACC]',
      r'\d{3}-\d{4}-\d{4}-\d{2}' : '[ACC]',
      r'\d{3}-\d{5}-\d{3}-\d{2}' : '[ACC]',
      r'\d{3}-\d{6}-\d{2}-\d{3}' : '[ACC]',
      r'\d{4}-\d{2,3}-\d{6,7}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}-\d{1}' : '[ACC]',
      r'\d{1}-\d{6}-\d{2,3}' : '[ACC]',
      r'(\d{2}|\d{6})-\d{2}-\d{6}' : '[ACC]',

      r'\d{3}-\d{4}-\d{4}' : '[TEL]',
      r'\d{2}-\d{3,4}-\d{4}' : '[TEL]'
  }

  sorted_patterns = sorted(numbers.items(), key=lambda x: len(x[0]), reverse=True)

  num = []
  matches = []
  already_matched = set()

  for pattern, token in sorted_patterns:
    for match in re.finditer(pattern, sentence):
      start = match.start()
      end = match.end()
      if all(pos not in already_matched for pos in range(start, end)):
        matches.append((match.start(), token))
        already_matched.update(range(start, end))


  matches.sort(key = lambda x : x[0])

  num = [token for _, token in matches]

  # 숫자 정보를 제외한 문자 정보만으로 input 만든 뒤 모델에 넣어 ---------------

  pattern = '|'.join(f'({key})' for key in numbers.keys())

  sentence_no_num = re.sub(pattern, '[PAD]', sentence) # 숫자 정보는 [PAD] 으로 처리

  tokenized_sentence = tokenizer.tokenize(sentence_no_num)

  sent_in_num = tokenizer.convert_tokens_to_ids(tokenized_sentence)

  input_sentence = pad_sequences([sent_in_num], maxlen = max_length, dtype = 'int',
                                    value = 0, truncating = 'post', padding = 'post')

  input_mask = [[1 if tok > 0 else 0 for tok in sent] for sent in input_sentence]

  input_sentence = torch.tensor(input_sentence).to(device)
  input_mask = torch.tensor(input_mask).to(device)

  with torch.no_grad():
    output = model(input_sentence, attention_mask = input_mask)
    logits = output.logits

  pred = np.argmax(logits.cpu().numpy(), axis = 2)

  tokens = tokenizer.convert_ids_to_tokens(input_sentence[0])

  # 다시 문장의 형태로 결합 ----------------------------------------------------

  out_sentence = []
  current_token = ""
  prev_tag = None
  tag_name = None

  puncs = [".",","]

  for token, tag_index in zip(tokens, pred[0]):
      tag_name = index_to_tag[tag_index][2:]

      if (token == '[PAD]'):
        if current_token:
          out_sentence.append(current_token)
          current_token = ""
        out_sentence.append('[PAD]')

      elif tag_name == 'PS':
        if token.startswith("##"):
          if tag_name == prev_tag:
            pass
          else:
            current_token += (f"[MASK-{tag_name}]")

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (f"[MASK-{tag_name}]")

      else: # 가릴 게 없다면
        if token in puncs:
          current_token += token

        elif token.startswith("##"):
          current_token += (token[2:])

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (token)



      prev_tag = tag_name

  out_sentence.append(current_token)

  out_sentence = ' '.join(out_sentence)

  # [PAD] -> 마스킹 토큰으로 ---------------------------------------------------

  out_sentence_split = out_sentence.split()

  new_sentence = []

  pad_count = 0

  for token in out_sentence_split:
    if token == '[PAD]':
      if pad_count < len(num):
        new_sentence.append(num[pad_count])
        pad_count += 1
    else:
      new_sentence.append(token)

  out_sentence = ' '.join(new_sentence)

  return(out_sentence)

In [ ]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
ner_pii_masking_loose(sentence, model, tokenizer, index_to_tag, max_length, device)

고양이 [MASK-PS]는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 [TEL] 이고, 주민등록번호는 [RRN] 이다. 또한, 우리 [ACC] 은행을 쓴다.


### ner_pii_masking_w_ast

----------------
< ner_pii_masking_w_ast >
- 다른 방식으로 마스킹 진행
- 모든 label 마스킹, * 로 출력
- 전화번호, 주민등록번호, 계좌번호의 경우 자릿수 맞춰서 asterisk로 표현 0
----------------

In [24]:
def ner_pii_masking_w_ast(sentence, model, tokenizer, index_to_tag, max_length, device):

  # 숫자 정보 마스킹 후 리스트 num에 저장 (순서대로) ---------------------------
  numbers = {
      r'\d{6}-\d{7}' : '[RRN]',

      r'\d{3}-\d{2}-\d{6,7}' : '[ACC]',
      r'\d{3}-\d{3}-(\d{6}|\d{8})' : '[ACC]',
      r'\d{3}-(\d{5}|\d{8})-\d{3}' : '[ACC]',
      r'\d{3}-\d{6}-\d{5}' : '[ACC]',
      r'\d{3}-\d{4}-\d{4}-\d{2}' : '[ACC]',
      r'\d{3}-\d{5}-\d{3}-\d{2}' : '[ACC]',
      r'\d{3}-\d{6}-\d{2}-\d{3}' : '[ACC]',
      r'\d{4}-\d{2,3}-\d{6,7}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}-\d{1}' : '[ACC]',
      r'\d{1}-\d{6}-\d{2,3}' : '[ACC]',
      r'(\d{2}|\d{6})-\d{2}-\d{6}' : '[ACC]',

      r'\d{3}-\d{4}-\d{4}' : '[TEL]',
      r'\d{2}-\d{3,4}-\d{4}' : '[TEL]'
  }

  sorted_patterns = sorted(numbers.items(), key=lambda x: len(x[0]), reverse=True)

  num = []
  matches = []
  already_matched = set()

  for pattern, token in sorted_patterns:
    for match in re.finditer(pattern, sentence):
      start = match.start()
      end = match.end()
      if all(pos not in already_matched for pos in range(start, end)):
        matches.append((match.start(), token))
        already_matched.update(range(start, end))


  matches.sort(key = lambda x : x[0])

  num = [token for _, token in matches]

  # 숫자 정보를 제외한 문자 정보만으로 input 만든 뒤 모델에 넣어 ---------------

  pattern = '|'.join(f'({key})' for key in numbers.keys())

  sentence_no_num = re.sub(pattern, '[PAD]', sentence) # 숫자 정보는 [PAD] 으로 처리

  tokenized_sentence = tokenizer.tokenize(sentence_no_num)

  sent_in_num = tokenizer.convert_tokens_to_ids(tokenized_sentence)

  input_sentence = pad_sequences([sent_in_num], maxlen = max_length, dtype = 'int',
                                    value = 0, truncating = 'post', padding = 'post')

  input_mask = [[1 if tok > 0 else 0 for tok in sent] for sent in input_sentence]

  input_sentence = torch.tensor(input_sentence).to(device)
  input_mask = torch.tensor(input_mask).to(device)

  with torch.no_grad():
    output = model(input_sentence, attention_mask = input_mask)
    logits = output.logits

  pred = np.argmax(logits.cpu().numpy(), axis = 2)

  tokens = tokenizer.convert_ids_to_tokens(input_sentence[0])

  # 다시 문장의 형태로 결합 ----------------------------------------------------

  out_sentence = []
  current_token = ""
  prev_tag = None
  tag_name = None
  puncs = [".",","]

  for token, tag_index in zip(tokens, pred[0]):
      if (token == '[PAD]'):
        if current_token:
          out_sentence.append(current_token)
          current_token = ""
        out_sentence.append('[PAD]')

      elif (tag_index == 0):

        if token in puncs:
          current_token += token

        elif token.startswith("##"):
          current_token += (token[2:])

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (token)

      elif (tag_index > 0):
        tag_name = index_to_tag[tag_index][2:]

        if token.startswith("##"):
          if tag_name == prev_tag:
            pass
          else:
            current_token += ("***")

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += ("***")

      prev_tag = tag_name

  out_sentence.append(current_token)

  out_sentence = ' '.join(out_sentence)

  # [PAD] -> 마스킹 토큰으로 ---------------------------------------------------

  out_sentence_split = out_sentence.split()

  new_sentence = []

  pad_count = 0

  ast = {'[TEL]':'***-****-****', '[RRN]':'******-*******', '[ACC]':'***-**-******'}
  for token in out_sentence_split:
    if token == '[PAD]':
      if pad_count < len(num):
        new_sentence.append(ast[num[pad_count]])
        pad_count += 1
    else:
      new_sentence.append(token)

  out_sentence = ' '.join(new_sentence)

  return(out_sentence)

In [25]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
ner_pii_masking_w_ast(sentence, model, tokenizer, index_to_tag, max_length, device)

'고양이 ***는 우리 집 막내이며, *** ***에 살고 있다. 전화번호는 ***-****-**** 이고, 주민등록번호는 ******-******* 이다. 또한, 우리 ***-**-****** 은행을 쓴다.'

## ppt 사진용 코드들

In [ ]:
def ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device):

  # 숫자 정보 마스킹 후 리스트 num에 저장 (순서대로) ---------------------------
  numbers = {
      r'\d{6}-\d{7}' : '[RRN]',

      r'\d{3}-\d{2}-\d{6,7}' : '[ACC]',
      r'\d{3}-\d{3}-(\d{6}|\d{8})' : '[ACC]',
      r'\d{3}-(\d{5}|\d{8})-\d{3}' : '[ACC]',
      r'\d{3}-\d{6}-\d{5}' : '[ACC]',
      r'\d{3}-\d{4}-\d{4}-\d{2}' : '[ACC]',
      r'\d{3}-\d{5}-\d{3}-\d{2}' : '[ACC]',
      r'\d{3}-\d{6}-\d{2}-\d{3}' : '[ACC]',
      r'\d{4}-\d{2,3}-\d{6,7}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}-\d{1}' : '[ACC]',
      r'\d{1}-\d{6}-\d{2,3}' : '[ACC]',
      r'(\d{2}|\d{6})-\d{2}-\d{6}' : '[ACC]',

      r'\d{3}-\d{4}-\d{4}' : '[TEL]',
      r'\d{2}-\d{3,4}-\d{4}' : '[TEL]'
  }

  sorted_patterns = sorted(numbers.items(), key=lambda x: len(x[0]), reverse=True)

  num = []
  matches = []
  already_matched = set()

  for pattern, token in sorted_patterns:
    for match in re.finditer(pattern, sentence):
      start = match.start()
      end = match.end()
      if all(pos not in already_matched for pos in range(start, end)):
        matches.append((match.start(), token))
        already_matched.update(range(start, end))


  matches.sort(key = lambda x : x[0])

  num = [token for _, token in matches]

  # 숫자 정보를 제외한 문자 정보만으로 input 만든 뒤 모델에 넣어 ---------------

  pattern = '|'.join(f'({key})' for key in numbers.keys())

  sentence_no_num = re.sub(pattern, '[PAD]', sentence) # 숫자 정보는 [PAD] 으로 처리

  tokenized_sentence = tokenizer.tokenize(sentence_no_num)

  sent_in_num = tokenizer.convert_tokens_to_ids(tokenized_sentence)

  input_sentence = pad_sequences([sent_in_num], maxlen = max_length, dtype = 'int',
                                    value = 0, truncating = 'post', padding = 'post')

  input_mask = [[1 if tok > 0 else 0 for tok in sent] for sent in input_sentence]

  input_sentence = torch.tensor(input_sentence).to(device)
  input_mask = torch.tensor(input_mask).to(device)

  with torch.no_grad():
    output = model(input_sentence, attention_mask = input_mask)
    logits = output.logits

  pred = np.argmax(logits.cpu().numpy(), axis = 2)

  tokens = tokenizer.convert_ids_to_tokens(input_sentence[0])

  # 다시 문장의 형태로 결합 ----------------------------------------------------

  out_sentence = []
  current_token = ""
  prev_tag = None
  tag_name = None

  puncs = [".",","]

  for token, tag_index in zip(tokens, pred[0]):
      if (token == '[PAD]'):
        if current_token:
          out_sentence.append(current_token)
          current_token = ""
        out_sentence.append('[PAD]')

      elif (tag_index == 0):
        if token in puncs:
          current_token += token

        elif token.startswith("##"):
          current_token += (token[2:])

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (token)

      elif (tag_index > 0):
        tag_name = index_to_tag[tag_index][2:]

        if token.startswith("##"):
          if tag_name == prev_tag:
            pass
          else:
            current_token += (f"[MASK-{tag_name}]")

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (f"[MASK-{tag_name}]")

      prev_tag = tag_name

  out_sentence.append(current_token)

  out_sentence = ' '.join(out_sentence)

  # [PAD] -> 마스킹 토큰으로 ---------------------------------------------------

  out_sentence_split = out_sentence.split()

  new_sentence = []

  pad_count = 0

  for token in out_sentence_split:
    if token == '[PAD]':
      if pad_count < len(num):
        new_sentence.append(num[pad_count])
        pad_count += 1
    else:
      new_sentence.append(token)

  out_sentence = ' '.join(new_sentence)

  return(out_sentence)

  print(f"이 문장에는 {num}의 숫자 데이터가 있습니다.")
  print(f"문장을 다시 결합하기 전에는 {tokens} 이렇게 생겼습니다.")

In [ ]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

In [35]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."

numbers = {
      r'\d{6}-\d{7}' : '[RRN]',

      r'\d{3}-\d{2}-\d{6,7}' : '[ACC]',
      r'\d{3}-\d{3}-(\d{6}|\d{8})' : '[ACC]',
      r'\d{3}-(\d{5}|\d{8})-\d{3}' : '[ACC]',
      r'\d{3}-\d{6}-\d{5}' : '[ACC]',
      r'\d{3}-\d{4}-\d{4}-\d{2}' : '[ACC]',
      r'\d{3}-\d{5}-\d{3}-\d{2}' : '[ACC]',
      r'\d{3}-\d{6}-\d{2}-\d{3}' : '[ACC]',
      r'\d{4}-\d{2,3}-\d{6,7}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}-\d{1}' : '[ACC]',
      r'\d{1}-\d{6}-\d{2,3}' : '[ACC]',
      r'(\d{2}|\d{6})-\d{2}-\d{6}' : '[ACC]',

      r'\d{3}-\d{4}-\d{4}' : '[TEL]',
      r'\d{2}-\d{3,4}-\d{4}' : '[TEL]'
  }

sorted_patterns = sorted(numbers.items(), key=lambda x: len(x[0]), reverse=True)

num = []
matches = []
already_matched = set()

for pattern, token in sorted_patterns:
    for match in re.finditer(pattern, sentence):
      start = match.start()
      end = match.end()
      if all(pos not in already_matched for pos in range(start, end)):
        matches.append((match.start(), token))
        already_matched.update(range(start, end))

matches.sort(key = lambda x : x[0])

num = [token for _, token in matches]

print(f"이번 문장은 '{sentence}' 입니다.")
print(f"이 문장에는 {num}의 숫자 정보가 있습니다.")

이번 문장은 '고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다.' 입니다.
이 문장에는 ['[TEL]', '[RRN]', '[ACC]']의 숫자 정보가 있습니다.


In [29]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
basic = ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device)
loose = ner_pii_masking_loose(sentence, model, tokenizer, index_to_tag, max_length, device)
ast = ner_pii_masking_w_ast(sentence, model, tokenizer, index_to_tag, max_length, device)
print(f"가장 기본 마스킹 결과: {basic}")
print(f"루즈한 마스킹 결과: {loose}")
print(f"*로 마스킹 결과: {ast}")

가장 기본 마스킹 결과: 고양이 [MASK-PS]는 우리 집 막내이며, [MASK-LC] [MASK-LC]에 살고 있다. 전화번호는 [TEL] 이고, 주민등록번호는 [RRN] 이다. 또한, 우리 [ACC] 은행을 쓴다.
루즈한 마스킹 결과: 고양이 [MASK-PS]는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 [TEL] 이고, 주민등록번호는 [RRN] 이다. 또한, 우리 [ACC] 은행을 쓴다.
*로 마스킹 결과: 고양이 ***는 우리 집 막내이며, *** ***에 살고 있다. 전화번호는 ***-****-**** 이고, 주민등록번호는 ******-******* 이다. 또한, 우리 ***-**-****** 은행을 쓴다.
